In [1]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import re 
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
!pip install seaborn
import seaborn as sns
matplotlib.rcParams['pdf.fonttype'] = 42
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

You are using pip version 9.0.3, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [12]:
df=pd.read_csv('skyscrapers.csv',encoding='latin-1')

,City,Country,All\nBuildings,100m+,150m+,200m+,300m+,Telecom\nTowers,All\nStructures
0,Hong Kong,China,923,644,355,66,6,0,923
1,New York City,United States,"1,549",861,281,80,11,2,"1,557"
2,Shenzhen,China,279,251,226,84,11,0,279
3,Dubai,United Arab Emirates,467,317,193,83,23,1,468
4,Shanghai,China,416,213,162,55,5,2,418


In [16]:
df.rename(columns={'All\nBuildings':'AllBuildings',
                   'Telecom\nTowers':'TelecomTowers',
                   'All\nStructures':'AllStructures'}, inplace=True)
df['AllBuildings'] = df['AllBuildings'].replace({'\,':''}, regex = True)
df['AllStructures'] = df['AllStructures'].replace({'\,':''}, regex = True)
df.head(10)

,City,Country,AllBuildings,100m+,150m+,200m+,300m+,TelecomTowers,AllStructures
0,Hong Kong,China,923,644,355,66,6,0,923
1,New York City,United States,1549,861,281,80,11,2,1557
2,Shenzhen,China,279,251,226,84,11,0,279
3,Dubai,United Arab Emirates,467,317,193,83,23,1,468
4,Shanghai,China,416,213,162,55,5,2,418
5,Tokyo,Japan,404,329,155,27,0,2,407
6,Chicago,United States,572,327,126,31,6,2,574
7,Chongqing,China,155,145,119,39,2,0,155
8,Guangzhou,China,175,137,115,35,10,1,177
9,Jakarta,Indonesia,410,153,91,40,0,1,411


# combine city value count with city gdp

In [23]:
df2=pd.read_csv(
    '/Users/tongxucai/Documents/datajournalism/fall/skyscrapers/city_GDP.csv',
    encoding='latin-1',
    na_values=[' '])
df2=df2[['City/Metropolitan area','Official est.\nNominal\nGDP ($BN)']]
df2.head(4)

,City/Metropolitan area,Official est.\nNominal\nGDP ($BN)
0,Aachen-LiÃ¨ge-Maastricht,NaN
1,Aberdeen,11.3 (2008)[5]
2,Abidjan,NaN
3,Abu Dhabi,119[6]


In [25]:
df2.rename(columns={'City/Metropolitan area':'City'}, inplace=True)

In [28]:
join=df.merge(df2, how='inner',on=['City'],left_index=True,right_index=True)
join

,City,Country,AllBuildings,100m+,150m+,200m+,300m+,TelecomTowers,AllStructures,Official est.\nNominal\nGDP ($BN)
0,Hong Kong,China,923,644,355,66,6,0,923,NaN
1,New York City,United States,1549,861,281,80,11,2,1557,11.3 (2008)[5]
2,Shenzhen,China,279,251,226,84,11,0,279,NaN
3,Dubai,United Arab Emirates,467,317,193,83,23,1,468,119[6]
4,Shanghai,China,416,213,162,55,5,2,418,NaN
5,Tokyo,Japan,404,329,155,27,0,2,407,59.1 (2015â16)[7]
6,Chicago,United States,572,327,126,31,6,2,574,NaN
7,Chongqing,China,155,145,119,39,2,0,155,37.300 (2016)[8]
8,Guangzhou,China,175,137,115,35,10,1,177,52.339 (2016)[8]
9,Jakarta,Indonesia,410,153,91,40,0,1,411,43.250 (2016)[8]


In [29]:
join.rename(columns={'Official est.\nNominal\nGDP ($BN)':'NominalGDP'}, inplace=True)
join

,City,Country,AllBuildings,100m+,150m+,200m+,300m+,TelecomTowers,AllStructures,NominalGDP
0,Hong Kong,China,923,644,355,66,6,0,923,NaN
1,New York City,United States,1549,861,281,80,11,2,1557,11.3 (2008)[5]
2,Shenzhen,China,279,251,226,84,11,0,279,NaN
3,Dubai,United Arab Emirates,467,317,193,83,23,1,468,119[6]
4,Shanghai,China,416,213,162,55,5,2,418,NaN
5,Tokyo,Japan,404,329,155,27,0,2,407,59.1 (2015â16)[7]
6,Chicago,United States,572,327,126,31,6,2,574,NaN
7,Chongqing,China,155,145,119,39,2,0,155,37.300 (2016)[8]
8,Guangzhou,China,175,137,115,35,10,1,177,52.339 (2016)[8]
9,Jakarta,Indonesia,410,153,91,40,0,1,411,43.250 (2016)[8]


In [30]:
join.to_csv('city_skyscrapers_GDP.csv')

In [82]:
chinacity=pd.read_csv('China_city_gdp.csv',encoding='latin-1')
chinacity.head(3)

,City,GDP
0,Shanghai,CNY 1641bn
1,Beijing,CNY 1521bn
2,Shenzhen,CNY 1213bn


In [83]:
chinacity['GDP']=chinacity['GDP'].str[3:]
chinacity['GDP']

0      1641bn
1      1521bn
2      1213bn
3      1176bn
4      1037bn
       ...   
95        0bn
96      127bn
97      127bn
98        0bn
99      121bn
Name: GDP, Length: 100, dtype: object

In [84]:
chinacity['GDP']=chinacity['GDP'].str[:-2]
chinacity.head(3)

,City,GDP
0,Shanghai,1641
1,Beijing,1521
2,Shenzhen,1213


In [85]:
addchinacitygdp=join.merge(chinacity, how='inner',on=['City'],left_index=True,right_index=True)
addchinacitygdp.head(3)

,City,Country,AllBuildings,100m+,150m+,200m+,300m+,TelecomTowers,AllStructures,NominalGDP,GDP
0,Hong Kong,China,923,644,355,66,6,0,923,NaN,1641
1,New York City,United States,1549,861,281,80,11,2,1557,11.3 (2008)[5],1521
2,Shenzhen,China,279,251,226,84,11,0,279,NaN,1213


In [86]:
addchinacitygdp=addchinacitygdp.drop('NominalGDP', axis=1)
addchinacitygdp

,City,Country,AllBuildings,100m+,150m+,200m+,300m+,TelecomTowers,AllStructures,GDP
0,Hong Kong,China,923,644,355,66,6,0,923,1641
1,New York City,United States,1549,861,281,80,11,2,1557,1521
2,Shenzhen,China,279,251,226,84,11,0,279,1213
3,Dubai,United Arab Emirates,467,317,193,83,23,1,468,1176
4,Shanghai,China,416,213,162,55,5,2,418,1037
5,Tokyo,Japan,404,329,155,27,0,2,407,1033
6,Chicago,United States,572,327,126,31,6,2,574,955
7,Chongqing,China,155,145,119,39,2,0,155,770
8,Guangzhou,China,175,137,115,35,10,1,177,748
9,Jakarta,Indonesia,410,153,91,40,0,1,411,695


In [91]:
addchinacitygdp['GDP']=addchinacitygdp['GDP'].astype(int)
addchinacitygdp['GDP']

0     1641
1     1521
2     1213
3     1176
4     1037
      ... 
95       0
96     127
97     127
98       0
99     121
Name: GDP, Length: 100, dtype: int64

In [92]:
addchinacitygdp['GDP']=addchinacitygdp['GDP']/7
addchinacitygdp['GDP']

0     234.428571
1     217.285714
2     173.285714
3     168.000000
4     148.142857
         ...    
95      0.000000
96     18.142857
97     18.142857
98      0.000000
99     17.285714
Name: GDP, Length: 100, dtype: float64

In [96]:
addchinacitygdp.to_csv('firsttry.csv')

# integrate chinese cities' gdp

In [48]:
china=pd.read_csv('chinacitygdp.csv')

In [49]:
china.head(2)

,Province,GDP_usd_millions,city
0,Macau,"50,559",Macau
1,Hong Kong,"341,679",Hong Kong


In [51]:
china.rename(columns={'city':'City'}, inplace=True)

In [52]:
china.head(2)

,Province,GDP_usd_millions,City
0,Macau,"50,559",Macau
1,Hong Kong,"341,679",Hong Kong


In [53]:
china.to_csv('chinacitygdp.csv')

In [55]:
firsttry=pd.read_csv('firsttry.csv')

In [95]:
addchina=pd.merge(firsttry,china, how='left',on='City')
addchina.head(5)

,City,Country,AllBuildings,100m+,150m+,200m+,300m+,400+,500+,600+,TelecomTowers,AllStructures,Province,GDP_usd_millions
0,Hong Kong,China,923.0,644.0,355.0,66.0,10,3.0,1.0,NaN,0.0,923.0,Hong Kong,"341,679"
1,Mecca,Saudi Arabia,7.0,7.0,7.0,7.0,2,2.0,1.0,1.0,NaN,7.0,NaN,NaN
2,Jeddah,Saudi Arabia,14.0,9.0,6.0,2.0,7,3.0,1.0,1.0,2.0,16.0,NaN,NaN
3,Riyadh,Saudi Arabia,39.0,27.0,17.0,9.0,6,7.0,NaN,NaN,0.0,39.0,NaN,NaN
4,New York City,United States,1549.0,861.0,281.0,80.0,30,8.0,1.0,NaN,2.0,1557.0,NaN,NaN


In [96]:
addchina.to_csv('firsttry.csv')

# add U.S. city GDP

In [58]:
US=pd.read_excel('US_city_gdp.xlsx')
US.head(5)

,Metropolitan area 2017,million_chained_2009_usd
0,"New York-Newark-Jersey City, NY-NJ-PA",1444484
1,"Los Angeles-Long Beach-Anaheim, CA",904899
2,"Chicago-Naperville-Elgin, IL-IN-WI",583137
3,"Dallas-Fort Worth-Arlington, TX",479678
4,"Washington-Arlington-Alexandria, DC-VA-MD-WV",460026


In [61]:
US['Metropolitan area 2017'].str[:-3]

0            New York-Newark-Jersey City, NY-NJ
1               Los Angeles-Long Beach-Anaheim,
2               Chicago-Naperville-Elgin, IL-IN
3                  Dallas-Fort Worth-Arlington,
4     Washington-Arlington-Alexandria, DC-VA-MD
                        ...                    
95                                   Anchorage,
96                       Cape Coral-Fort Myers,
97                                 Springfield,
98                               Stockton-Lodi,
99                              Chattanooga, TN
Name: Metropolitan area 2017, Length: 100, dtype: object

In [62]:
US['Metropolitan area 2017'] = US['Metropolitan area 2017'].replace({'\,':'-'}, regex = True)
US['Metropolitan area 2017']

0            New York-Newark-Jersey City- NY-NJ-PA
1               Los Angeles-Long Beach-Anaheim- CA
2               Chicago-Naperville-Elgin- IL-IN-WI
3                  Dallas-Fort Worth-Arlington- TX
4     Washington-Arlington-Alexandria- DC-VA-MD-WV
                          ...                     
95                                   Anchorage- AK
96                       Cape Coral-Fort Myers- FL
97                                 Springfield- MA
98                               Stockton-Lodi- CA
99                              Chattanooga- TN-GA
Name: Metropolitan area 2017, Length: 100, dtype: object

In [69]:
US['Metropolitan area 2017']=US['Metropolitan area 2017'].str.extract('^([A-Z][a-z]+\s?[A-Z]?[a-z]+)\-')
US['Metropolitan area 2017']

0        New York
1     Los Angeles
2         Chicago
3          Dallas
4      Washington
         ...     
95      Anchorage
96     Cape Coral
97    Springfield
98       Stockton
99    Chattanooga
Name: Metropolitan area 2017, Length: 100, dtype: object

In [70]:
US.head(10)

,Metropolitan area 2017,million_chained_2009_usd
0,New York,1444484
1,Los Angeles,904899
2,Chicago,583137
3,Dallas,479678
4,Washington,460026
5,Houston,436369
6,San Francisco,425358
7,Philadelphia,387217
8,Boston,379499
9,Atlanta,334488


In [72]:
US.to_csv('UScitygdpcleaned.csv')

In [74]:
UScity=pd.read_csv('UScitygdpcleaned.csv')
UScity.head(5)

,Metropolitan area 2017,million_chained_2009_usd
0,New York,1444484
1,Los Angeles,904899
2,Chicago,583137
3,Dallas,479678
4,Washington,460026


In [78]:
UScity.rename(columns={'Metropolitan area 2017':'City',
                       'million_chained_2009_usd':'GDP_usd_millions'}, inplace=True)
UScity.head(5)

,City,GDP_usd_millions
0,New York,1444484
1,Los Angeles,904899
2,Chicago,583137
3,Dallas,479678
4,Washington,460026


In [133]:
secondtry=pd.read_csv('firsttry.csv')

In [134]:
secondtry[["GDP_usd_millions","City"]][secondtry["Country"]=="United States"]

,GDP_usd_millions,City
4,NaN,New York City
9,NaN,Chicago
25,NaN,Miami
37,NaN,Houston
45,NaN,Los Angeles
48,NaN,San Francisco
54,NaN,Seattle
55,NaN,Boston
56,NaN,Dallas
61,NaN,Atlanta


In [136]:
tmp = secondtry.merge(UScity[["City","GDP_usd_millions"]],on="City",how="left")
tmp["GDP_usd_millions_x"][tmp["GDP_usd_millions_x"].isnull()] = tmp["GDP_usd_millions_y"][tmp["GDP_usd_millions_x"].isnull()]
tmp

/Users/tongxucai/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,City,Country,AllBuildings,100m+,150m+,200m+,300m+,400+,500+,600+,TelecomTowers,AllStructures,Province,GDP_usd_millions_x,GDP_usd_millions_y
0,0,Hong Kong,China,923.0,644.0,355.0,66.0,10,3.0,1.0,NaN,0.0,923.0,Hong Kong,"341,679",NaN
1,1,Mecca,Saudi Arabia,7.0,7.0,7.0,7.0,2,2.0,1.0,1.0,NaN,7.0,NaN,NaN,NaN
2,2,Jeddah,Saudi Arabia,14.0,9.0,6.0,2.0,7,3.0,1.0,1.0,2.0,16.0,NaN,NaN,NaN
3,3,Riyadh,Saudi Arabia,39.0,27.0,17.0,9.0,6,7.0,NaN,NaN,0.0,39.0,NaN,NaN,NaN
4,4,New York City,United States,1549.0,861.0,281.0,80.0,30,8.0,1.0,NaN,2.0,1557.0,NaN,NaN,NaN
5,5,Shenzhen,China,279.0,251.0,226.0,106.0,21,3.0,1.0,NaN,0.0,279.0,Guangdong,"333,097",NaN
6,6,Dubai,United Arab Emirates,467.0,317.0,193.0,83.0,32,5.0,1.0,1.0,1.0,468.0,NaN,NaN,NaN
7,7,Shanghai,China,416.0,213.0,162.0,55.0,12,5.0,1.0,1.0,2.0,418.0,Shanghai,"453,701",NaN
8,8,Tokyo,Japan,404.0,329.0,155.0,27.0,0,NaN,NaN,NaN,2.0,407.0,NaN,NaN,NaN
9,9,Chicago,United States,572.0,327.0,126.0,31.0,8,3.0,1.0,NaN,2.0,574.0,NaN,583137,583137.0


In [137]:
tmp.to_csv('secondtry.csv')

In [139]:
thirdtry=pd.read_csv('secondtry.csv')
thirdtry.Country.value_counts()

China                   37
United States           16
South Korea              5
Saudi Arabia             4
Japan                    4
Australia                3
Canada                   3
Philippines              3
United Arab Emirates     3
India                    2
Thailand                 2
Brazil                   2
France                   2
Vietnam                  2
Indonesia                1
North Korea              1
Argentina                1
Kuwait                   1
Russia                   1
Bahrain                  1
Qatar                    1
Mexico                   1
Singapore                1
Israel                   1
Colombia                 1
Panama                   1
United Kingdom           1
Turkey                   1
Malaysia                 1
Germany                  1
Name: Country, dtype: int64